In [5]:
import galarp as grp
import numpy as np

from astropy import units as u

from gala.units import galactic

In [50]:
class UniformGridParticleSet(grp.ParticleSet):

    def __init__(self, Rmax = 10 * u.kpc, spacing = 0.5 * u.kpc, 
                 disp_R = 0 * u.km / u.s, disp_z = 0 * u.km/u.s,  
                 units=galactic, **kwargs):
        super().__init__(units=units, **kwargs)
        
        self.Rmax = Rmax.to(units["length"]).value
        self.spacing = spacing.to(units["length"]).value

        self.disp_R = disp_R.to(self.unitset.velocity).value    
        self.disp_z = disp_z.to(self.unitset.velocity).value

    def seed(self, potential, **kwargs):
        xs, ys = np.meshgrid(np.arange(-self.Rmax, self.Rmax, self.spacing),
                             np.arange(-self.Rmax, self.Rmax, self.spacing))
        xs = xs.flatten()
        ys = ys.flatten()

        # Remove particles outside of Rmax
        rs = np.sqrt(xs**2 + ys**2)
        mask = rs <= self.Rmax
        xs = xs[mask]
        ys = ys[mask]

        zs = np.zeros_like(xs)

        self.positions = np.vstack((xs, ys, zs))

        self.gen_velocities(potential, **kwargs)

        # Add velocity dispersion to the disk if requested
        self.velocities[0] += np.random.normal(0, self.disp_R, self.velocities.shape[1])
        self.velocities[1] += np.random.normal(0, self.disp_R, self.velocities.shape[1])
        self.velocities[2] += np.random.normal(0, self.disp_z, self.velocities.shape[1])


pset = UniformGridParticleSet(Rmax=10*u.kpc, spacing=0.2*u.kpc, disp_R=10*u.km/u.s, disp_z=2*u.km/u.s)
pset.seed(potential=grp.builtins.satpots.JZ2023_Satellite())

In [49]:
sim = grp.RPSim(satellite_potential=grp.builtins.satpots.JZ2023_Satellite(),
                particles=pset)
orbits = sim.run(wind_on=False, integration_time=1000 * u.Myr)

grp.plotting.k3d_plot([orbits])

Output()